<a href="https://colab.research.google.com/github/thatsciencegal/flood_pheno/blob/main/SASA_PEAN_cc_rem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialize Google Earth Engine

In [ ]:
import ee

## Trigger the authentication flow
ee.Authenticate()

## Initialize the library
ee.Initialize()

## Test the API by printing elevation of Mt. Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Import the Landsat 8 data for the area between São Salvador and Peixe Angical

In [ ]:
## Import the Landsat 8 data for the area between São Salvador and Peixe Angical
l8SasaPean = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
          .filter(ee.Filter.eq('WRS_PATH', 222))\
          .filter(ee.Filter.eq('WRS_ROW', 69))\
          .filterDate('2017-07-01', '2018-06-30')
print('Collection: ')
l8SasaPean.getInfo()